<a href="https://colab.research.google.com/github/BrandanWood/COMP593/blob/Lab-3/Copy_of_WEEK_3_WORKING_WITH_DATABASES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 3: WORKING WITH DATABASES


## Introduction

During this lab, we will experiment with using a few different libararies and modules to connect to and interact with an SQLite database. After creating and configuring our testing database, we will populate it with Fake data and run some queries to see what kind of information we can retreive. For context, we will be creating RSSN, the "Really Simple Social Network".

## Creating a SQLite Database

We will be utilizing the `sqlite3` library to create our database file, as it offers the convienience of creating our database file automatically in the event that it cannot find an existing file at the path specified.

**Before you run the below code:**
On the left of the notebook, select the file folder icon. Then, after you run the code block, you should be able to observe the creation of the database file. If it doesn't appear, try clicking the "Refresh" button above the list of folders.

In [3]:
import sqlite3

#When we retreive a Connection object, a new database will be created for us if it doesn't already exist. 
myConnection = sqlite3.connect('social_network.db')
print(sqlite3.version)

2.6.0


## Creating a Table


We'll use the below code to create a table called `people` within our database. 

In [28]:
import sqlite3

#Retreive the Connection object
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Let's define the SQL Query we will use to create our first table:
createPeopleTable = """ CREATE TABLE IF NOT EXISTS people (
                          name text NOT NULL,
                          email text NOT NULL,
                          address text NOT NULL,
                          city text NOT NULL,
                          province text NOT NULL,
                          country text NOT NULL,
                          phone text,
                          bio text,
                          dob date NOT NULL,
                          heatmap integer,
                          created_at datetime NOT NULL,
                          updated_at datetime NOT NULL,
                          ipv4 text
                        );"""

#Now that we have the string to create our table,
#Cursor objects have an execute() method which will accept an SQL string and perform the operations described.

myCursor.execute(createPeopleTable)

#We can confirm if our table was created successfully by running the following SQL Query
#pragma_table_info is an internal SQLite function that will retun information about a table
myCursor.execute("SELECT group_concat(name, ', ') FROM pragma_table_info('people')")
print(myCursor.fetchone())

#We use to the commit() method on the database Connection object to persist our changes
myConnection.commit()

#It is always a good idea to close a connection when it will no longer be used
myConnection.close()


('name, email, address, city, province, country, phone, bio, dob, heatmap, created_at, updated_at, ipv4',)



If you received a tuple containing the names of the columns, awesome! We have successfully created our database table.
```
('id, name, email, address, city, province, country, phone, bio, created_at, updated_at',)
```

Run the below code block to add our first entry.

In [18]:
# REMOVED THIS SCRIPT BECAUSE IT IS NO LONGER NEEDED TO RUN LAB SCRIPT

# Lab Submission

We're going to build our experience with working with Libraries and examining documentation by populating our 'People' table with data provided to us by the `Faker` library. `Faker` is used to generate fake data and is very helpful for the rapid generation of databases for the purposes of testing. Run the two blocks below to install faker and get an idea of what it can do.

In [2]:
!pip install faker

     |████████████████████████████████| 1.5 MB 3.2 MB/s 


In [13]:
from faker import Faker

fake = Faker()

x = 2
y = "bob"

s = "{} is {}".format(x,y)
print(s)

for _ in range(10):
  print('{} is a {}'.format( fake.name(), fake.job() ) )

2 is bob
Julian Burns is a Print production planner
Anthony Hernandez is a Scientist, water quality
Lisa Thompson is a Equality and diversity officer
Hannah Robles is a Legal secretary
Jason Roach is a Graphic designer
Karen Thomas is a Embryologist, clinical
Lisa Daniels is a Engineer, mining
Kelly Avery is a Advertising account planner
Roberto Smith is a Manufacturing engineer
Julia Johnson is a Animator


Very cool! Faker has tons of `providers` that can all be used to populate fake data. The [list of providers](https://faker.readthedocs.io/en/stable/providers.html) in the Faker documentation will help you fill out the columns for our People table.

The goal of this script is to populate the people table with 1000 entries, with the following constraints:

1. The `heatmap` column must contain a random number between `999` and `2500`
2. The `created_at` and `updated_at` columns must use the `datetime` object (see examples above) 
2. Use `Faker` to generate all other fields.

*Hint: Each of the methods contained in the provider can be called directly from the base Faker object, for example, one can call the `file_name()` method from `faker.providers.file` by calling `Faker().file_name()`*

In [29]:
import sqlite3
import pprint
from faker import Faker
from datetime import datetime
from random import randint

# Connecting to the DB
conn = sqlite3.connect('social_network.db', timeout=0)

# cursor with Query
c = conn.cursor()
addPersonQuery = """INSERT INTO people (
                      name, 
                      email, 
                      address, 
                      city, 
                      province, 
                      country, 
                      phone, 
                      bio,
                      dob,
                      heatmap,
                      created_at, 
                      updated_at, 
                      ipv4)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

fake = Faker()
# list of provinces and x rand value based on list len - 1
randProvince = ("Alberta", "British Columbia", "Manitoba", "New Brunswick", "Newfoundland and Labrador",
                "Northwest Territories", "Nova Scotia", "Nunavut", "Ontario", "Prince Edward Island",
                "Quebec", "Saskatchewan", "Yukon")



# testing random provinces


# for loop that contains the stuff for fake generation of people
for stuff in range(1000):
  x = randint(0, len(randProvince) - 1)
  print(randProvince[x])
  myPerson = (
              fake.name(),
              fake.ascii_free_email(),
              fake.street_address(),
              fake.city(),
              randProvince[x],
              fake.country(),
              fake.phone_number(),
              fake.sentence(nb_words=10),
              fake.date_of_birth(),
              fake.random_int(min=999, max=2500),
              datetime.now(),
              datetime.now(),
              fake.ipv4_public()
  )
  c.execute(addPersonQuery, myPerson)
  print(myPerson)

# commits the changes to the DB file
conn.commit()




conn.close()

Yukon
('Patrick Barber', 'jamesfreeman@hotmail.com', '749 Richard Wells', 'Sandraborough', 'Yukon', 'Macao', '426.408.8018x77581', 'Institution practice specific price last oil capital finish.', datetime.date(1983, 3, 26), 2070, datetime.datetime(2022, 2, 17, 19, 49, 33, 586455), datetime.datetime(2022, 2, 17, 19, 49, 33, 586457), '109.59.66.180')
Newfoundland and Labrador
('Alex Mcpherson', 'john77@yahoo.com', '18222 Andrew Rue Suite 165', 'Reginaland', 'Newfoundland and Labrador', 'Bermuda', '001-400-924-7076x3212', 'Successful size budget sea arrive gun challenge eight.', datetime.date(1984, 12, 26), 1207, datetime.datetime(2022, 2, 17, 19, 49, 33, 592623), datetime.datetime(2022, 2, 17, 19, 49, 33, 592625), '207.22.208.114')
Yukon
('John Morris', 'butlerjames@yahoo.com', '6804 Morgan Row Suite 210', 'Lake Sarah', 'Yukon', 'Svalbard & Jan Mayen Islands', '001-338-721-0109x260', 'Maintain site again company media none human take trip old chance suddenly.', datetime.date(1953, 2, 21),

Finally, the last step,
Using the code block below, combined with what you have learned above, and the Lecture notes,
Craft a SQL Query that will return the `name` of no more than `20` `people` with a `heatmap` greater than `1500`

In [47]:
import sqlite3
import pprint

pp = pprint.PrettyPrinter()
#Retreive the Connection object
conn = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
c = conn.cursor()
c.execute("""SELECT name, heatmap FROM people WHERE heatmap > 1500 LIMIT 20
""")
results = c.fetchall()


pp.pprint(results)
conn.close()

[('Patrick Barber', 2070),
 ('John Morris', 2115),
 ('Taylor Chambers', 1593),
 ('Kimberly Zuniga', 1844),
 ('Kathleen Lara DDS', 1906),
 ('David Fisher DDS', 1748),
 ('Jodi Holder', 2250),
 ('Bryan Chan', 2373),
 ('Lisa Martin', 1842),
 ('Brandon Cruz', 2459),
 ('Carolyn Guzman', 1777),
 ('Juan Goodwin', 2046),
 ('Douglas Patton', 1662),
 ('Tiffany Hamilton', 1886),
 ('Kenneth Miller', 1670),
 ('Mrs. Carolyn Harper', 1819),
 ('Jeremy Nguyen', 2024),
 ('David Jones', 1661),
 ('Mr. Clayton Gomez', 2090),
 ('Tyler Bishop', 1857)]


Your submission will contain, as usual, a link to your completed colab notebook, but in addition to that, you will download a copy of your social_network.db file and upload it to D2L. To download the file, right click it from the Files menu on the left of the Notebook.